# Continusing processing without Pint

In [14]:
import polars as pl
import numpy as np
import pathlib
import re

In [15]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Mild-Steel-Tempering':
    print("Path is projregexct current working directory to the project root")

Path is projregexct current working directory to the project root


In [16]:
resources_path = pathlib.PurePath(pathlib.PurePath(cwd), 'resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Mild-Steel-Tempering/resources')

In [17]:
data_path = f"{resources_path}/AZoM_scraped_properties.csv"
df_scraped = pl.read_csv(data_path)
df_scraped.glimpse()

Rows: 17
Columns: 10
$ steel         <str> 'AISI 1090', 'AISI-SAE 5140', 'AISI-SAE 4140', 'AISI-SAE 1045', 'AISI-SAE 1050', 'AISI-SAE 1035', 'AISI 6145', 'AISI-SAE 1038', 'AISI-SAE 5160', 'AISI 4640'
$ density       <str> '7.85', '7.85', '7.85', '7.87', '7.85', '7.85', '7.75', '7.845', '7.85', '7.75'
$ units_density <str> 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3', 'g/cm3'
$ pr            <str> '0.27-0.30', '0.27-0.30', '0.27-0.30', '0.29', '0.27-0.30', '0.27-0.30', '0.27-030', '0.27-0.30', '0.27 – 0.30', '0.27-030'
$ em            <str> '190-210', '190-210', '190-210', '200', '190-210', '190-210', '201-209', '190-210', '190-210', '201-209'
$ em_unit       <str> 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa', 'GPa'
$ ys            <i64> 540, 295, 415, 450, 580, 370, 1165, 485, 275, 1103
$ ys_unit       <str> 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa', 'MPa'
$ uts           <i64> 696, 570, 655, 585, 690, 585, 1213, 5

In [18]:
df_scraped.select(['units_density', 'em_unit', "ys_unit", "uts_unit"]).unique()

units_density,em_unit,ys_unit,uts_unit
str,str,str,str
"""g/cc""","""GPa""","""MPa""","""MPa"""
"""g/cm3""","""GPa""","""MPa""","""MPa"""


# Drop units columns are they are constant for each column

Convert each column name to include the units

All units in units_density are a 1:1 conversion with g/cm3

g/mL (water) = g/cc = g/cm3 = grams per cubic centimeter

In [19]:
df_scraped_unit_header = df_scraped.select(['steel', 
                                            'density', 
                                            'pr', 
                                            'em', 
                                            'ys',
                                            'uts']).rename({'density' : 'density_g_per_cm3',
                                                                        'em' : 'em_GPa',
                                                                        'ys' : 'ys_MPa',
                                                                        'uts' : 'uts_MPa'})

print(df_scraped_unit_header.shape)    
df_scraped_unit_header.head()

(17, 6)


steel,density_g_per_cm3,pr,em_GPa,ys_MPa,uts_MPa
str,str,str,str,i64,i64
"""AISI 1090""","""7.85""","""0.27-0.30""","""190-210""",540,696
"""AISI-SAE 5140""","""7.85""","""0.27-0.30""","""190-210""",295,570
"""AISI-SAE 4140""","""7.85""","""0.27-0.30""","""190-210""",415,655
"""AISI-SAE 1045""","""7.87""","""0.29""","""200""",450,585
"""AISI-SAE 1050""","""7.85""","""0.27-0.30""","""190-210""",580,690


In [20]:
df_scraped_unit_header.describe()

statistic,steel,density_g_per_cm3,pr,em_GPa,ys_MPa,uts_MPa
str,str,str,str,str,f64,f64
"""count""","""17""","""17""","""17""","""17""",17.0,17.0
"""null_count""","""0""","""0""","""0""","""0""",0.0,0.0
"""mean""",null,null,null,null,526.058824,698.176471
"""std""",null,null,null,null,245.556122,217.899127
"""min""","""AISI 1030""","""7.7-8.03""","""0.27 – 0.30""","""190-210""",275.0,515.0
"""25%""",null,null,null,null,415.0,570.0
"""50%""",null,null,null,null,470.0,655.0
"""75%""",null,null,null,null,540.0,696.0
"""max""","""AISI-SAE 5160""","""7.87""","""0.29""","""201-209""",1165.0,1276.0


### Many values have leading zero but no decimal resulting in parsing as whole number not decimal value

In [21]:
def if_no_decimal_point(num):
    #If leading 0 followed by number then replace leading 0 with "0."
    regex = r"^0\d"
    if re.search(regex, num):
        #replace the 
        num = re.sub("^0", "0.", num)
    return np.float64(num)

def strip_string(string):
    return string.strip()


In [22]:
#Clean density column
df_scraped_clean = df_scraped_unit_header.clone()
s_split = df_scraped_clean['density_g_per_cm3'].str.split('-')
s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))
s_mean_d = s_numeric_l.list.mean()

df_scraped_clean.with_columns(density_g_per_cm3 = s_meand)

/tmp/ipykernel_90183/1643061001.py:4: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))


steel,density_g_per_cm3,pr,em_GPa,ys_MPa,uts_MPa
str,f64,str,str,i64,i64
"""AISI 1090""",7.85,"""0.27-0.30""","""190-210""",540,696
"""AISI-SAE 5140""",7.85,"""0.27-0.30""","""190-210""",295,570
"""AISI-SAE 4140""",7.85,"""0.27-0.30""","""190-210""",415,655
"""AISI-SAE 1045""",7.87,"""0.29""","""200""",450,585
"""AISI-SAE 1050""",7.85,"""0.27-0.30""","""190-210""",580,690
…,…,…,…,…,…
"""AISI 1074 Carbon Steel""",7.865,"""0.27-0.30""","""190-210""",505,650
"""AISI-SAE 4340""",7.85,"""0.27-0.30""","""190-210""",470,745
"""AISI-SAE 4027""",7.85,"""0.27-0.30""","""190-210""",325,515


In [23]:

# AISI-SAE 5160 and AISI-SAE 6150 use a different dash than the others for the pr range
# Discovered when 2 nulls showed up after processing
s_same_dashes = df_scraped_clean["pr"].str.replace('–', '-')
s_split = s_same_dashes.str.split('-')
s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))
s_mean_pr = s_numeric_l.list.mean()


/tmp/ipykernel_90183/304290526.py:5: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))


In [24]:
s_same_dashes = df_scraped_clean["em_GPa"].str.replace('–', '-')
s_split = s_same_dashes.str.split('-')
s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))
s_mean_em = s_numeric_l.list.mean()

/tmp/ipykernel_90183/1369062398.py:3: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))


In [26]:

df_scraped_clean = df_scraped_clean.with_columns(density_g_per_cm3 = s_mean_d,
                                                 poissons_ratio = s_mean_pr, 
                                                 elastic_modulus_GPa = s_mean_em,
                                                 yield_strength_MPa = df_scraped_unit_header['ys_MPa'],
                                                 ultimate_strength_MPA = df_scraped_unit_header['uts_MPa'])\
                                                    .select(['steel', 
                                                             "density_g_per_cm3", 
                                                             "poissons_ratio" , 
                                                             "elastic_modulus_GPa", 
                                                             "yield_strength_MPa", 
                                                             "ultimate_strength_MPA"])
df_scraped_clean.describe()

statistic,steel,density_g_per_cm3,poissons_ratio,elastic_modulus_GPa,yield_strength_MPa,ultimate_strength_MPA
str,str,f64,f64,f64,f64,f64
"""count""","""17""",17.0,17.0,17.0,17.0,17.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",null,7.84,0.285294,200.588235,526.058824,698.176471
"""std""",null,0.034415,0.001213,1.660528,245.556122,217.899127
"""min""","""AISI 1030""",7.75,0.285,200.0,275.0,515.0
"""25%""",null,7.85,0.285,200.0,415.0,570.0
"""50%""",null,7.85,0.285,200.0,470.0,655.0
"""75%""",null,7.85,0.285,200.0,540.0,696.0
"""max""","""AISI-SAE 5160""",7.87,0.29,205.0,1165.0,1276.0


In [27]:
df_scraped_clean.write_csv(f'{resources_path}/AZoM_scraped_properties.csv')

# Iterations on regex

In [187]:
s_split = df_scraped_unit_header['poissons_ratio'].str.split('-')

def if_no_decimal_point(num):
    #If leading 0 followed by number then replace leading 0 with "0."
    regex = r"^0\d"
    if re.search(regex, num):
        #replace the 
        num = re.sub("^0", "0.", num)
    return np.float64(num)

s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))


/tmp/ipykernel_26553/2808776221.py:11: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  s_numeric_l = s_split.map_elements(lambda x: list(map(if_no_decimal_point, x)))


In [174]:
def if_no_decimal(num):
    #If leading 0 followed by number then replace leading 0 with "0."
    regex = r"^0\d"
    if re.search(regex, num):
        #replace the 
        num = re.sub("^0", "0.", num)
    return np.float64(num)

lst = ["030","0.40",".5","13"]

list(map(if_no_decimal, lst))

[np.float64(0.3), np.float64(0.4), np.float64(0.5), np.float64(13.0)]

In [168]:
regex = r"^0\d"

for string in ["040","0.40","40","04",".4"]:
    print(re.search(regex, string))


<re.Match object; span=(0, 2), match='04'>
None
None
<re.Match object; span=(0, 2), match='04'>
None


# Pint does not functionn as desired for units with a denominator and the documentation is out of date. 

Scrapping that section of the work. Units can be done manually

In [45]:
!pip install pandas pint_pandas

In [10]:
import pathlib

In [92]:
# New imports
import pandas as pd
import pint
import pint_pandas
pint_pandas.show_versions()
ureg = pint.UnitRegistry()
pint.UnitRegistry().default_system = 'cgs'

{'numpy': '2.1.1', 'pandas': '2.2.3', 'pint': '0.24.3', 'pint_pandas': '0.6.2'}


In [11]:
cwd = pathlib.Path.cwd()

if cwd.name == 'Mild-Steel-Tempering':
    print("Path is project root")
else:
    print("Please correct current working directory to the project root")

Path is project root


In [12]:
resources_path = pathlib.PurePath(pathlib.PurePath(cwd), 'resources')
resources_path

PurePosixPath('/home/mox/Documents/coding_projects/bootcamp_local/Homeworks/Mild-Steel-Tempering/resources')

In [16]:
data_path = f"{resources_path}/scraped_properties.csv"
df_data = pd.read_csv(data_path)
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   steel            28 non-null     object 
 1   density          28 non-null     object 
 2   units_density    28 non-null     object 
 3   poissons_ratio   28 non-null     object 
 4   elastic_modulus  28 non-null     float64
 5   units_em         28 non-null     object 
dtypes: float64(1), object(5)
memory usage: 1.4+ KB


In [19]:
df_data['units_density'].unique()

array(['g/cm3', 'g/cc', 'g/'], dtype=object)

In [20]:
df_data['units_em'].unique()

array(['g/cm3'], dtype=object)

# Clean units

Unnecessary step all units are variations of grams per cubic centimeter

This step is included to provide a guideline on how to unify units if there were different scales or to conversions to be done

## pint can not be used for unit type conversion in polars. 

Pandas must be used. 

All units in units_density are a 1:1 conversion with g/cm3

g/mL (water) = g/cc = g/cm3 = grams per cubic centimeter

In [156]:
# pint does not recognize cm3 a a unit. cc is used. could define cm3 as cc
df_data['units_density'] = "g/cm3"

In [157]:
df_data['units_density'].unique()

array(['g/cm3'], dtype=object)

In [158]:
g_per_cc = ureg.g / ureg.cc
g_per_cc

<Unit('gram / cubic_centimeter')>

In [99]:
ureg.define('density = g / cc')

In [100]:
pint_density = df_data['density'].astype('pint[density]')
pint_density

UndefinedUnitError: 'density' is not defined in the unit registry

In [93]:
pint_vol = 500 * ureg.cc
pint_vol

<Quantity(500, 'cubic_centimeter')>

In [94]:
pint_mass = pint_density * pint_vol

ValueError: Cannot operate with Quantity and Quantity of different registries.

## Demonstrate a unit conversion using pint

g to kg should divide the values by 1000

then cc to L should multiply by 1000

In [84]:
pint_density.pint.to("kg / cc")

TypeError: can't multiply sequence by non-int of type 'float'

# Experimenting with pint and pandas


In [55]:

df = pd.DataFrame({
    "density": pd.Series([1, 2, 2, 3], dtype="pint[g / cc]"),
    "angular_velocity": pd.Series([1, 2, 2, 3], dtype="pint[cc]"),
})
df



,density,angular_velocity
0,1,1
1,2,2
2,2,2
3,3,3


In [65]:

df = pd.DataFrame({
    "density": pd.Series([1, 2, 2, 3], dtype="pint[g / cc]"),
    "vol": pd.Series([1, 2, 2, 3], dtype="pint[cc]"),
})
df


,density,vol
0,1,1
1,2,2
2,2,2
3,3,3


In [61]:
df['grams'] = df['density'] * df['vol']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype                        
---  ------   --------------  -----                        
 0   density  4 non-null      pint[gram / cubic_centimeter]
 1   vol      4 non-null      pint[cubic_centimeter]       
 2   grams    4 non-null      pint[gram]                   
dtypes: pint[cubic_centimeter](1), pint[gram / cubic_centimeter](1), pint[gram](1)
memory usage: 240.0 bytes
